In [ ]:
from sentence_transformers import SentenceTransformer,util
import json
import lawquery
import string
import pandas as pd


In [ ]:
import chromadb
from chromadb.config import Settings
from chromadb.utils import embedding_functions
sentence_transformer_ef = embedding_functions.InstructorEmbeddingFunction(device="cuda",model_name="hkunlp/instructor-xl")


chroma_client = chromadb.Client(Settings(
    chroma_db_impl="duckdb+parquet",
    persist_directory="chroma",
))
chroma_client.reset()
collection = chroma_client.get_or_create_collection(name="law_documents",embedding_function=sentence_transformer_ef)

In [ ]:
data = json.load(open('data.json'))
engine = lawquery.Engine('./documents/luat-bao-hiem-xa-hoi-2014/tree.json.gz')

documents = []
metadatas = []
ids = []

for row in data['article']:
    result = engine.query(node_type=row['node_type'], node_id=row['node_id'])[0]
    text = result.content

    documents.append(text)
    metadatas.append({'node_id': result.node_id, 'node_type': result.node_type, })
    ids.append(result.id)
    collection.add(documents=[text],metadatas=[{'node_id': result.node_id, 'node_type': result.node_type, }],ids=[result.id])
# collection.add(documents=documents,metadatas=metadatas,ids=ids)

In [ ]:
qa_df = pd.read_json('./answers.jsonl', lines=True)
qa_df.head()

In [ ]:
law_id = '58/2014/QH13'
new_df = qa_df[qa_df['answers'].apply(lambda x: any(law_id in y for y in x))]

In [ ]:
random_row = new_df.sample(1).iloc[0]
print(random_row['answers'])
collection.query(
    query_texts=[random_row['cauhoi']],
    n_results=5,
    include=['metadatas'],
)